In [2]:
from my_functions import*

In [3]:
file= uproot.open("/home/ruben/Escritorio/BachelorThesisRuben/Data/Muon_trees.root") #opening the Root file with Uproot 

In [4]:
MuonTree_ZeroBias=file["MuonTree_ZeroBias;1"]
MuonTree_Zmumu=file["MuonTree_Zmumu;1"]

#Select range of events
nmin=0
nmax=2000

#Select quality 0 Z->mumu
Zmumu_pt=quality_selector(MuonTree_Zmumu["muon_quality"].array(),MuonTree_Zmumu["muon_pt"].array(),0)[nmin:nmax]
Zmumu_eta=quality_selector(MuonTree_Zmumu["muon_quality"].array(),MuonTree_Zmumu["muon_eta"].array(),0)[nmin:nmax]
Zmumu_phi=quality_selector(MuonTree_Zmumu["muon_quality"].array(),MuonTree_Zmumu["muon_phi"].array(),0)[nmin:nmax]

#And select the Z peak pairs
Zmumu_pt, Zmumu_eta, Zmumu_phi = get_all_Z_peak_pairs(Zmumu_pt,Zmumu_eta,Zmumu_phi)

#Select the ZeroBias data with energy cut
ZeroBias_eta=energy_cut(MuonTree_ZeroBias["muon_pt"].array(), MuonTree_ZeroBias["muon_eta"].array())[nmin:nmax]
ZeroBias_phi=energy_cut(MuonTree_ZeroBias["muon_pt"].array(), MuonTree_ZeroBias["muon_phi"].array())[nmin:nmax]
ZeroBias_pt=energy_cut(MuonTree_ZeroBias["muon_pt"].array(), MuonTree_ZeroBias["muon_pt"].array())[nmin:nmax]

quality_selector: Only 21.265 % of the data has been selected
quality_selector: Only 21.265 % of the data has been selected
quality_selector: Only 21.265 % of the data has been selected


100%|██████████| 2000/2000 [00:01<00:00, 1262.80it/s]


energy_cut: Only 5.79% of the events have survived the cut with a lower cut of 14000 and no upper cut
energy_cut: Only 5.79% of the events have survived the cut with a lower cut of 14000 and no upper cut
energy_cut: Only 5.79% of the events have survived the cut with a lower cut of 14000 and no upper cut


In [17]:
def ROC_FPR_efficiency(MuonTree_Zmumu, MuonTree_ZeroBias, Zmumu_pt, Zmumu_eta, Zmumu_phi, ZeroBias_pt, ZeroBias_eta, ZeroBias_phi,
                 event_range: tuple = [0,5000], dr_min: float = 0.05, dr_max: float=0.3, target_efficiency: float=0.9,
                   bins: list = np.linspace(0,1,1000)):
    """
    Computes the False Positive Rate (FPR) at a given target signal efficiency (TPR)
    for the isolation ROC curves between Zmumu and ZeroBias events.

    Inputs:
    -MuonTree_Zmumu : tree containing Zmumu data.
    -MuonTree_ZeroBias : tree containing ZeroBias data.
    -Zmumu_pt/eta/phi or ZeroBias_pt/eta/phi : array containing transverse mometum, eta and phi for Zmumu or ZeroBias
    -event_range : tuple of int, range of event indices to use (default is [0, 5000]).
    -dr_min : float, minimum delta R used in the ROC computation (default is 0.05).
    -dr_max : float, maximum delta R used in the ROC computation (default is 0.3).
    -target_efficiency : float, signal efficiency (TPR) to evaluate FPR at (default is 0.9).
    -bins : list, bins for the histogram used in ROC computation (default is linspace from 0 to 1 with 1000 bins).

    Returns:
    -FPR_eff : float, False Positive Rate at the specified target signal efficiency.
    dr_min : float, minimum delta R value used in computation.
    dr_max : float, maximum delta R value used in computation (for reference).
    """
    nmin, nmax= event_range
    ROC_values, _, _ = compute_ROC_curve(MuonTree_Zmumu, MuonTree_ZeroBias, Zmumu_pt, Zmumu_eta, Zmumu_phi, ZeroBias_pt, ZeroBias_eta, ZeroBias_phi,
                            [nmin,nmax],[nmin,nmax], bins, [dr_min], [dr_max])
    FPR=ROC_values[0][0]
    TPR=ROC_values[0][1]
    FPR_eff=np.min(FPR[TPR >= target_efficiency])

    return FPR_eff, dr_min, dr_max

In [44]:
def ROC_FPR_efficiencies(MuonTree_Zmumu, MuonTree_ZeroBias, Zmumu_pt, Zmumu_eta, Zmumu_phi, ZeroBias_pt, ZeroBias_eta, ZeroBias_phi,
                 event_range: tuple = [0,5000], dr_min_range: tuple = [0.0,0.2], dr_max_range: tuple=[0.25,0.45],
                 steps: int = 4, target_efficiency: float=0.9, bins: list = np.linspace(0,1,1000)):
    """
    Reliying on the ROC_FPR_efficieny function, computes and returns the FPR for a given TPR efficieny for a given set of delta R values

    Inputs:
    -MuonTree_Zmumu : tree containing Zmumu data.
    -MuonTree_ZeroBias : tree containing ZeroBias data.
    -Zmumu_pt/eta/phi or ZeroBias_pt/eta/phi : array containing transverse mometum, eta and phi for Zmumu or ZeroBias
    -event_range : tuple of int, range of event indices to use (default is [0, 5000]).
    -dr_min : list of floats, minimum delta R range used in the ROC computation 
    -dr_max : list of floats, maximum delta R range used in the ROC computation 
    -target_efficiency : float, signal efficiency (TPR) to evaluate FPR at (default is 0.9).
    -bins : list, bins for the histogram used in ROC computation (default is linspace from 0 to 1 with 1000 bins).

    Returns:
    -FPR_effs : list of floats, False Positive Rates at the specified target signal efficiency.
    dr_min : float, minimum delta R values used in computation.
    dr_max : float, maximum delta R values used in computation (for reference).
    """
    dr_mins=np.linspace(dr_min_range[0], dr_min_range[1], steps)
    dr_maxs=np.linspace(dr_max_range[0], dr_max_range[1], steps)
    FPR_effs=[]
    for i in range(steps):
        for j in range(len(dr_maxs)):
            dr_min, dr_max = dr_mins[i], dr_maxs[j]
            FPR_eff, _, _=ROC_FPR_efficiency(MuonTree_Zmumu, MuonTree_ZeroBias, Zmumu_pt, Zmumu_eta, Zmumu_phi, ZeroBias_pt, ZeroBias_eta, ZeroBias_phi,
            event_range, dr_min, dr_max, target_efficiency, bins)
            FPR_effs.append(FPR_eff)
    FPR_effs = np.array(FPR_effs).reshape((steps, steps))
    return FPR_effs, dr_mins, dr_maxs

In [43]:
ROC_FPR_efficiencies(MuonTree_Zmumu, MuonTree_ZeroBias, Zmumu_pt, Zmumu_eta, Zmumu_phi, ZeroBias_pt, ZeroBias_eta, ZeroBias_phi, event_range=[0,2000])

(array([[0.85517241, 0.87311178],
        [0.80536913, 0.87410072]]),
 array([0. , 0.2]),
 array([0.25, 0.45]))

In [56]:
def ROC_FPR_2D_plot(MuonTree_Zmumu, MuonTree_ZeroBias, Zmumu_pt, Zmumu_eta, Zmumu_phi, ZeroBias_pt, ZeroBias_eta, ZeroBias_phi,
                 event_range: tuple = [0,5000], dr_min_range: tuple = [0.0,0.2], dr_max_range: tuple=[0.25,0.45],
                 steps: int = 4, target_efficiency: float=0.9, bins: list = np.linspace(0,1,1000)):
    """
    Plots a scatterplot containing the FPR(target_efficiency) values for a given dr_min, dr_max grid, relying on the ROC_FPR_efficiencies function 

    Inputs:
    -MuonTree_Zmumu : tree containing Zmumu data.
    -MuonTree_ZeroBias : tree containing ZeroBias data.
    -Zmumu_pt/eta/phi or ZeroBias_pt/eta/phi : array containing transverse mometum, eta and phi for Zmumu or ZeroBias
    -event_range : tuple of int, range of event indices to use (default is [0, 5000]).
    -dr_min : list of floats, minimum delta R range used in the ROC computation 
    -dr_max : list of floats, maximum delta R range used in the ROC computation 
    -target_efficiency : float, signal efficiency (TPR) to evaluate FPR at (default is 0.9).
    -bins : list, bins for the histogram used in ROC computation (default is linspace from 0 to 1 with 1000 bins).

    Returns:
    -Plot
    """
    FPR_effs, dr_mins, dr_maxs= ROC_FPR_efficiencies(MuonTree_Zmumu, MuonTree_ZeroBias, Zmumu_pt, Zmumu_eta, Zmumu_phi, ZeroBias_pt, ZeroBias_eta, ZeroBias_phi,
                         event_range, dr_min_range, dr_max_range, steps, target_efficiency, bins)
    
    x=dr_mins
    y=dr_maxs

    x,y = np.meshgrid(x,y)
    # Create scatter plot with color representing values
    plt.scatter(x, y, c=FPR_effs, cmap='viridis', s=200)  # s controls dot size
    plt.colorbar(label='FPR at TPR = {:.2f}'.format(target_efficiency))
    plt.xlabel('dr_min')
    plt.ylabel('dr_max')
    plt.title('FPR at TPR = {:.2f} over ΔR Ranges'.format(target_efficiency))
    plt.grid(True)
    plt.show()

In [49]:
ROC_FPR_2D_plot(MuonTree_Zmumu, MuonTree_ZeroBias, Zmumu_pt, Zmumu_eta, Zmumu_phi, ZeroBias_pt, ZeroBias_eta, ZeroBias_phi, steps=2)

Computing ROC curve:   0%|          | 0/1 [00:00<?, ?it/s].21s/it]